In [2]:
import requests
from bs4 import BeautifulSoup
import pickle

In [6]:
!pip install requests

In [9]:
def url_to_transcript(url):
    """Returns transcript data specifically from scrapsfromtheloft.com"""
    page = requests.get(url).text
    soup = BeautifulSoup(page,"lxml")
    text = [p.text for p in soup.find(class="elementor-element elementor-element-74af9a5b elementor-widget elementor-widget-theme-post-content").find_all('p')]
    print(url)
    return text

# URLs of transcript in space
urls = ["http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/",
       "http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/",
       "http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/",
       "http://scrapsfromtheloft.com/2017/08/07/bo-burnham-2013-full-transcript/",
       "http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/",
       "http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/",
       "http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/",
       "http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/",
       "http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/",
       "http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/",
       "http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/",
       "http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/"]

# Comedian names
comedians = ["louis","dave","ricky","bo","bill","jim","john","hasan","ali","anthony","mike","joe"]


SyntaxError: invalid syntax (2719477080.py, line 5)

In [4]:
transcripts = [url_to_transcript(u) for u in urls]

NameError: name 'urls' is not defined

In [10]:
!mkdir transcripts

for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt","wb") as file:
        pickle.dumb(transcripts[i], file)


mkdir: transcripts: File exists


NameError: name 'comedians' is not defined

In [11]:
data = {}
for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt","rb") as file:
        data[c] = pickle.load(file)

NameError: name 'comedians' is not defined

In [12]:
data.keys()

dict_keys([])

In [13]:
data["louis"][:2]

KeyError: 'louis'

# Cleaning the Data

#Common Steps:

- Make text all lower case
- Remove punctuation
- Remove numerical values
- Remove common non-sensical text (/n)
- Tokenize text
- Remove stop words
- Stemming / lemmatization
- Parts of speech tagging
- Create bi-grams or tri-grams
- Deal with typos

In [14]:
def combine_text(list_of_text):
    """Takes a list of text and combines them into one large chunk of text."""
    combined_text=" ".join(list_of_text)
    return combined_text

In [15]:
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [ ]:
import pandas as pd
pd.set_option("max-coldwith",150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ["transcript"]
data_df = data_df.sort_index()
data_df

In [ ]:
data_df.transcript.loc["ali"]

In [ ]:
import re 
import string

def clean_text_round1(text):
    text= text.lower()
    text= re.sub("\[.*?\]","",text)
    text= re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text= re.sub("\w*\d\w*","",text)
    text= re.sub("[""'']","",text)
    text= re.sub("\n","",text)
    return text
    
round1 = lambda x: clean_text_round1(x)

In [ ]:
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

In [ ]:
data_df.to_pickle("corpus.pkl")

# Document-Term Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words = "english")
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(),columns=cv.get_feature_names())
data_dtm.index = data_clean.index()
data_dtm

In [ ]:
data_dtm.to_pickel("dtm.pkl")

In [ ]:
data_clean.to_pickle("data_clean.pkl")
pickle.dump(cv, open("cv.pkl","wb"))